In [ ]:
# pip install pydantic

In [ ]:
# pip install boto3

In [ ]:
# pip install gspread

In [ ]:
# pip install openai

In [1]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work/agents


In [2]:
import yaml
import boto3
from core.task_publisher import TaskPublisher


ENV = "dev"
tenant_id = "alfa"

with open(f"../config/tenants/{tenant_id}/{ENV}.yaml", "r") as f:
    tenant_config = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {})
dynamodb = boto3.resource("dynamodb", region_name=dynamodb_config.get("region"))
messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))

sqs_config = tenant_config.get("sqs", {})
sqs = boto3.client("sqs", region_name=sqs_config.get("region"))
task_publisher = TaskPublisher(sqs, sqs_config.get("url"))

In [3]:
from core.processes.client_bank_statement import (
    ClientBankStatementProcess
)


processes_definitions = {
    ClientBankStatementProcess.PROCESS_TYPE: ClientBankStatementProcess,
}

In [4]:
from core.process_engine import ProcessEngine


engine = ProcessEngine(
    processes_table=processes_table,
    task_publisher=task_publisher,
    process_definitions=processes_definitions,
)

In [ ]:
from agents.hello_world.agent_factory import HelloWorldAgentFactory

context = {}

agent = HelloWorldAgentFactory.build(
    tenant_config=tenant_config,
    messages_table=messages_table,
    process_engine=engine,
)

result = agent.handle(context)
print("Agent result:", result)

tenant_id alfa
[PrintService to:525571969848] Hola Juan, espero que estés bien. Solo quería recordarte que tu estado sigue pendiente. Por favor, avísame si necesitas ayuda para avanzar. ¡Saludos cordiales!
Agent result: [{'client': {'id': '1', 'name': 'Juan', 'phone': '525571969848', 'status': 'pending'}, 'history': [{'direction': 'in', 'text': 'Hola'}, {'direction': 'out', 'text': 'Hola, como te puedo ayudar?'}, {'direction': 'in', 'text': 'Probando'}, {'direction': 'in', 'text': 'Probando 2'}, {'direction': 'in', 'text': '<non-text message>'}, {'direction': 'in', 'text': '<non-text message>'}, {'direction': 'in', 'text': '<non-text message>'}, {'direction': 'in', 'text': '<non-text message>'}, {'direction': 'in', 'text': 'Cómo está el clima?'}, {'direction': 'out', 'text': 'Hola Juan, espero que estés bien. Te recordamos que tienes un estado pendiente. Quedamos atentos para cualquier consulta. ¡Saludos cordiales!'}, {'direction': 'out', 'text': 'Hola Juan, espero que estés bien. Solo